<a href="https://colab.research.google.com/github/yding25/Kuka_with_Sucker/blob/main/display_objects_pybullet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Display all objects
<img src="https://raw.githubusercontent.com/yding25/pic_share/master/utensil.jpeg" height="300" />

Object library: <a href="https://github.com/yding25/URDF_models"> URDF_models </a>

# Setup

In [ ]:
#@title Download URDF models {display-mode: "form"}
# !rm -rf /content/urdf_models/
!git clone https://github.com/yding25/urdf_models.git

In [ ]:
#@title Import third-party packages {display-mode: "form"}
!pip install pybullet imageio-ffmpeg
import os
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pybullet
import pybullet_data
import cv2
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML
from google.colab.patches import cv2_imshow
from IPython.display import display
from tqdm.notebook import tqdm
from google.colab import files

In [ ]:
#@title Utils {display-mode: "form"}

class Client():
  def __init__(self):
    pybullet.connect(pybullet.DIRECT) # pybullet.GUI for local GUI.
    pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
    pybullet.setGravity(0, 0, -9.8)
    
    self.plane_id = pybullet.loadURDF("plane.urdf")

    # camera width and height
    self.cam_width = 4800
    self.cam_height = 4800

  def render_image(self):
    # camera parameters
    cam_target_pos = [1.0, 0.0, 0.5]
    cam_distance = 2.0
    cam_yaw, cam_pitch, cam_roll = -90, -90, 0
    cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 1], 2, 0.01, 100, 60
    cam_view_matrix = pybullet.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
    cam_projection_matrix = pybullet.computeProjectionMatrixFOV(cam_fov, self.cam_width*1./self.cam_height, cam_near_plane, cam_far_plane)
    znear, zfar = 0.01, 10.

    # get raw data
    _, _, color, depth, segment = pybullet.getCameraImage(
        width=self.cam_width,
        height=self.cam_height,
        viewMatrix=cam_view_matrix,
        projectionMatrix=cam_projection_matrix,
        shadow=1,
        flags=pybullet.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX,
        renderer=pybullet.ER_BULLET_HARDWARE_OPENGL)

    # get color image.
    color_image_size = (self.cam_width, self.cam_height, 4)
    color = np.array(color, dtype=np.uint8).reshape(color_image_size)
    color = color[:, :, :3]  # remove alpha channel

    # get depth image.
    depth_image_size = (self.cam_width, self.cam_height)
    zbuffer = np.float32(depth).reshape(depth_image_size)
    depth = (zfar + znear - (2 * zbuffer - 1) * (zfar - znear))
    depth = (2 * znear * zfar) / depth

    # get segment image.
    segment = np.reshape(segment, [self.cam_width, self.cam_height]) * 1. / 255.
    return color, depth, segment

  def reset_video(self):
    video = imageio_ffmpeg.write_frames('video.mp4', (self.cam_width, self.cam_height), fps=60)
    video.send(None) # seed the video writer with a blank frame
    return video

  def render_video(self, video, image):
    video.send(np.ascontiguousarray(image))
    
  def play_video(self):
    mp4 = open('video.mp4', 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)

  def add_object(self, utensil_name, utensil_pose):
    flags = pybullet.URDF_USE_INERTIA_FROM_FILE
    utensil_id = pybullet.loadURDF(utensil_name, basePosition=utensil_pose[0], baseOrientation=utensil_pose[1], flags=flags)
    return utensil_id
  
  def remove_object(self, utensil_id):
    pybullet.removeBody(utensil_id)

  def get_bounding_box(self, obj_id):
    (min_x, min_y, min_z), (max_x, max_y, max_z)= pybullet.getAABB(obj_id)
    return [min_x, min_y, min_z], [max_x, max_y, max_z]

  def home_joints(self):
    jointPositions = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    for jointIndex in range(pybullet.getNumJoints(self.robot_id)):
        pybullet.resetJointState(self.robot_id, jointIndex, jointPositions[jointIndex])
        pybullet.setJointMotorControl2(self.robot_id, jointIndex, pybullet.POSITION_CONTROL, jointPositions[jointIndex], 0)

  def disconnect(self):
    pybullet.disconnect()
  
  def sample_pose(self):
    # max_pose = [[1.5, 0.75, 0.6], [0, 0, 0, 1]]
    # min_pose = [[0.5, -0.75, 0.6], [0, 0, 0, 1]]
    pose_list = []
    for i in range(5):
      temp = [[1.4 - 0.3 * i, 0.75, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    for i in range(5):
      temp = [[1.4 - 0.3 * i, 0.45, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    for i in range(5):
      temp = [[1.4 - 0.3 * i, 0.15, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    for i in range(5):
      temp = [[1.4 - 0.3 * i, -0.15, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    for i in range(5):
      temp = [[1.4 - 0.3 * i, -0.45, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    for i in range(5):
      temp = [[1.4 - 0.3 * i, -0.75, 0.2], [0, 0, 0, 1]]
      pose_list.append(temp)
    return pose_list

# Initialize Environment

In [ ]:
#@title Initialize env {display-mode: "form"}
demo = Client()

In [ ]:
#@title add objects {display-mode: "form", vertical-output: true }
path_urdf_models = '/content/urdf_models/'
object_list = os.listdir(path_urdf_models)
counter = 0
utensil_list = []
for object in object_list: # remove noise
  if 'chair' in object:
    utensil_list.append(object)
utensil_list.sort()

# get potential pose
pose_list = demo.sample_pose()

index = 0
for object in utensil_list: # find path of object's urdf
  print('object: {}'.format(object))
  path_object = os.path.join(path_urdf_models, object)
  for file in os.listdir(path_object):
    if file.endswith('.urdf'):
      path_object_urdf = os.path.join(path_object, file)
      # print('path of object urdf: {}'.format(path_object_urdf))
      # add object
      utensil_name = path_object_urdf
      utensil_pose = pose_list[index]
      utensil_id = demo.add_object(path_object_urdf, utensil_pose)
      index += 1

object: furniture_chair


In [ ]:
#@title display object {display-mode: "form", vertical-output: true }
rgb, depth, mask = demo.render_image()
# plt.imshow(rgb)
plt.imsave('furniture.jpeg', rgb)
files.download('furniture.jpeg')
demo.disconnect() 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>